# CE888 Assignment 2 for Demo

This code is written to demonstarte a classifier using an autoencoder for ce888 assignment 2.

**Author**          : Tomoko Ayakawa<br> 
**Created on**      : 29 March 2019<br> 
**Last modified on**: 19 April 2019<br> 

In [ ]:
# import open source libraries
#import pandas as pd
#import seaborn as sns
import numpy as np
#import matplotlib.pyplot as plt
import sys
import keras

# import original libraries
sys.path.append("../")
sys.path.append("./sub")

#import utility as UTL
#import assignment2_main as MAIN2
from conf import myVariables as VAR
import load_data as DATA
import preprocess as PREP
import autoencoder as AE
import mlp as MLP
import grid_search as GS
import comparison as CMP

## 1. Prepare the data
### 1-1. Select the data to load
> 0. Human Activity
1. Spam
2. Phishing
>

In [ ]:
data_list={0: "human activity", 1: "spam", 2: "phishing"}
data_id=int(input(data_list))

col_names, features_df, targets_df, data_df, pic_file=DATA.load_data(data_id=data_id)
unique_labels=DATA.verify_data(data_df, targets_df, dispaly_range=False)

### 1-2. Obtain the small data

In [ ]:
features, classes=PREP.get_small_data(features_df, targets_df)

### 1-3. Pre-process the data
Fit the scaler and transform the data.<br>
When the scaler ID is not specified, MinMaxScaler will be applied.

In [ ]:
scl, features_nrm=PREP.pre_processing(features, display_result=False)

### 1-4. Split the data

In [ ]:
X, y=features_nrm, classes
X_tr, X_te, y_tr, y_te=PREP.split_data(features_nrm, classes)

## 4. Build an Autoencoder
### 4-1. Define the parameters

In [ ]:
ae_layers, mode, act, opt, loss, dropout, \
    epochs, verbose, summary_display=AE.get_parameters(data_id)

### 4-2. Train the autoencoder

In [ ]:
encoder, histories=AE.autoencoder(X, layers=ae_layers, mode=mode, act=act, opt=opt, 
                                 loss=loss, dropout=dropout, epochs=epochs, 
                                 verbose=verbose, summary_display=summary_display)

### 4-3. Display the training loss history

In [ ]:
AE.plot_ae_loss_history(histories, mode, pic_file)

### 4-4. Extract features from the datasets

In [ ]:
X_all_cmp=encoder.predict(X)
X_tr_cmp=encoder.predict(X_tr)
X_te_cmp=encoder.predict(X_te)

print("The number of compressed features:", len(X_all_cmp[0]))

## 5. Build a discriminative neural network
### 5-1. Define the parameters

finetune, h_num, h_act, out_act, opt, loss, epochs, val_rate, \
    verbose, summary_display=MLP.get_parameters()

### 5-2. Train the MLP

In [ ]:
k=ae_layers[-1]
n=len(np.unique(y))

model=MLP.build_mlp(encoder, num_in=k, num_out=n, \
            finetune=finetune, h_num=h_num, h_act=h_act, \
            out_act=out_act, opt=opt, loss=loss, \
            summary_display=summary_display)
histories=MLP.train_mlp(X, y, model, epochs=epochs, \
            val_rate=val_rate, verbose=verbose)

### 5-3. Display the training loss history

In [ ]:
MLP.plot_mlp_loss_history(histories, pic_file)

## 6. Grid Search for optimal parameter
### 6-1. Define the parameters
The codes and markdown below are originally written by the author for the assignment for CE802 which was submitted to the CSEE Department on 16 January 2019.<br><br>
Major parameters to be tuned are:


|parameter               |default    |description                                              |
|------------------------|-----------|---------------------------------------------------------|
|hidden_layer_sizes      |(100,)     |i-th element: numb of neurons in the i-th hidden layer   |
|activation              |"relu"     |activation function                                      |
|solver                  |"adam"     |weight optimasation                                      |
|alpha                   |0.0001     |regularization parameter                                 |
|learning_rate_init      |0.001      |step-size in weights update (solver=’sgd’ or ‘adam’)     |
|max_iter                |200        |max num of iterations                                    |
|momentum                |0.9        |momentum for gradient descent update (range (0, 1), solver=’sgd’|

In [ ]:
act, h_num, max_itr, lr, mmt, alpha, solver, splits=GS.get_parameters()

### 6-2. Grid Search

In [ ]:
param_grid, clf=GS.parameter_grid(activation=act, \
                           alpha=alpha, hidden_layer_sizes=(h_num,), \
                           max_iter=max_itr, learning_rate_init=lr, \
                           momentum=mmt, solver=solver)
if param_grid!=1: GS.grid_search(X_all_cmp, y, clf, param_grid, grid_splits=splits)

## 6. Evaluation
### 6-1. Cross Validation

In [ ]:
accs, fscores=MLP.cross_validation(model, X, y, epochs=epochs)

### 6-2. Train, predict and evaluate the classifier

In [ ]:
pred=MLP.evaluation(model, X_tr, y_tr, X_te, y_te, pic_file)

## 7. Comparison with other thechniques
### 7-1. Plot 3 dimensions obtained by PCA and Autoencoder

In [ ]:
# the number of unirue values in each dimension
for x, t in zip([X, X_all_cmp], ["PCA    ", "Encoder"]):
    print(t, ":", len(x), 
          "\t| axis-1:", len(np.unique(x[:,0])), 
          "\t| axis-2:", len(np.unique(x[:,1])),
          "\t| axis-3:", len(np.unique(x[:,2])))

In [ ]:
%matplotlib notebook
if ae_layers[-1]==3: CMP.plot_3D(X, X_all_cmp, y)

### 7-2. Performance comparison with DT, NB, SVM, NN

In [ ]:
%matplotlib inline
# original data (uncompressed)
CMP.performance_comparison(pred, X_tr, X_te, y_tr, y_te, data_id)

In [ ]:
# compressed data
CMP.performance_comparison(pred, X_tr_cmp, X_te_cmp, y_tr, y_te, pic_file, data_id)

# ===== End of the code =====